# Supporting Simple GraphRAG: Part 2 - Context Retrieval and Answer Generation

To run this Jupyter Notebook, you can download the original `.ipynb` file from [simple_graphrag_2.ipynb](https://github.com/tigergraph/tigergraphx/tree/main/docs/graphrag/simple_graphrag_2.ipynb).

---

## Workflow Overview

The query process of GraphRAG follows this workflow. TigerGraphX serves as the key interface, facilitating interaction with the embedding model to generate embeddings for user queries, retrieving context from TigerGraph, and communicating with the LLM chat model to generate answers.

![](https://raw.githubusercontent.com/tigerGraph/tigergraphx/refs/heads/main/docs/images/graphrag/querying.png)

---

## Get the Graph from TigerGraph
Since the graph has already been created in TigerGraph, redefining its schema is unnecessary. Instead, you can provide the graph name to retrieve it. TigerGraphX will verify if the graph exists in TigerGraph and, if it does, will return the corresponding graph.

### Define the TigerGraph Connection Configuration
Before retrive the graph schema from TigerGraph, you shoulddefine the  TigerGraph Connection Configuration first.
The recommended approach is to use environment variables, such as setting them with the `export` command in the shell. Here, to illustrate the demo, we configure them within Python using the `os.environ` method. You can find more methods for configuring connection settings in [Graph.\_\_init\_\_](../reference/01_core/graph.md#tigergraphx.core.graph.Graph.__init__).

In [1]:
>>> import os
>>> os.environ["TG_HOST"] = "http://127.0.0.1"
>>> os.environ["TG_USERNAME"] = "tigergraph"
>>> os.environ["TG_PASSWORD"] = "tigergraph"

### Retrieve the Graph
Once a graph has been created in TigerGraph, you can retrieve it without manually defining the schema using the `Graph.from_db` method, which requires only the graph name:

In [2]:
>>> from tigergraphx import Graph
>>> G = Graph.from_db("RetailGraph")

---
## Context Building: Writing Custom Context Builders

Context builders play a vital role in graph-powered RAG workflows. They transform retrieved graph data into structured, meaningful contexts for tasks such as interactions with LLMs.

TigerGraphX simplifies this process by offering the flexible `BaseContextBuilder` class, which allows developers to define custom logic for context building. You can find the source code in [this link](https://github.com/TigerGraph-DevLabs/tigergraphx/blob/main/tigergraphx/graphrag/base_context_builder.py).

In this example, we create a simple `ContextBuilder` by implementing the `build_context` method to define how the context is constructed. The process involves:  

- Retrieving the top **k** relevant objects (products) using TigerGraph's vector search.  
- For each retrieved product, fetching its properties and edges.  
- Concatenating all the gathered information into a structured context and returning it.  

In [3]:
>>> from typing import List
>>> from tigergraphx.graphrag import BaseContextBuilder

>>> class ContextBuilder(BaseContextBuilder):
...     async def build_context(self, query: str, k: int = 10) -> str | List[str]:
...         """Build local context."""
...         context: List[str] = []
... 
...         # Retrieve top-k objects
...         top_k_objects: List[str] = await self.retrieve_top_k_objects(
...             query, k=k, oversample_scaler=1
...         )
...         if not top_k_objects:
...             return ""  # Return early if no objects are retrieved
... 
...         # Iterate over all products
...         for product in top_k_objects:
...             node_data = self.graph.get_node_data(product, "Product")
...             edges = self.graph.get_node_edges(product, "Product")
...             context.append(f"Node Data for {product}: {node_data}")
...             context.append(f"Edges for {product}: {edges}")
... 
...         return "\n\n".join(context)

2025-03-28 15:15:41,490 - datasets - INFO - PyTorch version 2.6.0 available.


Here’s how you can utilize the custom context builder to integrate it with TigerGraphX and OpenAI components:  

- **Configure Vector Search:** The `vector_db` settings specify that the search engine will use **TigerVector** to retrieve relevant **Product** nodes based on their `emb_features` attribute.  
- **Set Up OpenAI Components:** The configuration defines models for **LLM**, **embedding**, and **chat**, ensuring that API keys are securely managed via environment variables.  
- **Initialize OpenAI and Search Engine:** The `create_openai_components` function instantiates the OpenAI chat model and the vector-based search engine for retrieving relevant nodes.  
- **Create Context Builder:** The `ContextBuilder` is initialized with the graph and search engine, allowing it to retrieve and structure context dynamically for downstream applications.

In [4]:
>>> from tigergraphx.factories import create_openai_components
>>> settings = {
...     "vector_db": {
...         "type": "TigerVector",
...         "graph_name": "RetailGraph",
...         "node_type": "Product",
...         "vector_attribute_name": "emb_features",
...     },
...     "llm": {
...         "type": "OpenAI",
...         # NOTE: The api_key must be provided via the environment variable OPENAI_API_KEY
...     },
...     "embedding": {
...         "type": "OpenAI",
...         "model": "text-embedding-3-small",
...     },
...     "chat": {
...         "type": "OpenAI",
...         "model": "gpt-4o-mini",
...     },
... }
>>> (openai_chat, search_engine) = create_openai_components(settings, G)
>>> context_builder = ContextBuilder(graph=G, search_engine=search_engine)

Let's verify that the context builder functions as expected.

In [5]:
>>> context = await context_builder.build_context("I am looking for a begginer drone. Please give me some recommendations.")
>>> print(context)

2025-03-28 15:15:46,803 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Node Data for SnapShot Voyager Remote Shutter Release: {'name': 'SnapShot Voyager Remote Shutter Release', 'price': 24.99, 'weight': 40, 'features': "Reduced Camera Shake: Allows you to trigger the shutter without touching the camera, minimizing movement and ensuring sharper images. Convenient Control: Provides a comfortable and ergonomic grip with a simple button to activate the shutter. Long Cable: Offers flexibility and ease of use with a generous cable length. Easy Connection: Plugs directly into the camera's remote port for quick and simple setup."}

Edges for SnapShot Voyager Remote Shutter Release: [('SnapShot Voyager Camera 3.0', 'SnapShot Voyager Remote Shutter Release'), ('SnapShot Voyager Remote Shutter Release', 'SnapShot Voyager Camera 3.0'), ('SnapShot Voyager Remote Shutter Release', 'Photography')]

Node Data for Aura X5 Pro: {'name': 'Aura X5 Pro', 'price'

Congratulations! It works.  

---  

## Generate Answer  

The final step is to generate an answer using the LLM.  

### Define a Prompt  
First, let's define a prompt template for the LLM to generate an answer as follows.

In [6]:
>>> PROMPTS = """---Role---
... 
... You are a helpful assistant responding to questions about data in the tables provided.
... 
... 
... ---Goal---
... 
... Generate a response of the target length and format that responds to the user's question, summarizing all information in the input data tables appropriate for the response length and format, and incorporating any relevant general knowledge.
... If you don't know the answer, just say so. Do not make anything up.
... Do not include information where the supporting evidence for it is not provided.
... 
... ---Target response length and format---
... 
... {response_type}
... 
... 
... ---Data tables---
... 
... {context_data}
... 
... 
... ---Goal---
... 
... Generate a response of the target length and format that responds to the user's question, summarizing all information in the input data tables appropriate for the response length and format, and incorporating any relevant general knowledge.
... 
... If you don't know the answer, just say so. Do not make anything up.
... 
... Do not include information where the supporting evidence for it is not provided.
... 
... 
... ---Target response length and format---
... 
... {response_type}
... 
... Add sections and commentary to the response as appropriate for the length and format. Style the response in markdown.
... """

### Define a Function to Generate Answer

Then let's define a `query` function that integrates the context builder and OpenAIChat to process user queries. This function retrieves relevant context from the graph, constructs a system prompt, and generates a response using the LLM.

In [7]:
>>> from tigergraphx.graphrag import BaseContextBuilder
>>> from tigergraphx.llm import OpenAIChat
>>> async def query(
...     query: str,
...     openai_chat: OpenAIChat,
...     context_builder: BaseContextBuilder,
...     top_k: int = 10,
...     only_need_context: bool = False,
...     response_type: str = "Multiple Paragraphs",
... ) -> str:
...     """
...     Perform a local search using the context builder and return the result.
...     """
...     # Generate context using the local context builder
...     context = await context_builder.build_context(
...         query,
...         k=top_k,
...     )
...     if only_need_context:
...         if not isinstance(context, str):
...             raise TypeError("Expected `context` to be an instance of str.")
...         return context
... 
...     # Validate that context exists
...     if not context:
...         return "Apologies, but I couldn't provide an answer as no relevant context was found for your query."
... 
...     # Construct the system prompt using the context
...     system_prompt = PROMPTS.format(context_data=context, response_type=response_type)
... 
...     # Perform the query using OpenAIChat
...     try:
...         response = await openai_chat.chat(
...             [
...                 {"role": "system", "content": system_prompt},
...                 {"role": "user", "content": query},
...             ]
...         )
...         return response
...     except Exception:
...         return "An error occurred while processing the query."

### Run the Function to Generate an Answer  
Great job! The final step is to provide a question and call the function to generate an answer.

In [8]:
>>> result = await query(
...     query="I am looking for a begginer drone. Please give me some recommendations.",
...     openai_chat=openai_chat,
...     context_builder=context_builder,
...     only_need_context=False,
... )
>>> print(result)

2025-03-28 15:15:53,971 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-03-28 15:16:04,309 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Recommendations for Beginner Drones

When it comes to selecting a beginner-friendly drone, it's essential to consider ease of use, durability, and features that enhance the flying experience. Based on available data, here are two excellent options for beginner drones:

#### 1. SkyHawk Zephyr Drone

**Price:** $0 (likely indicating it comes free with a package or promotional offer)

**Features:**
- **Simple Controls:** This drone is designed with beginner-friendly and intuitive controls, plus it includes automatic takeoff and landing capabilities to ease new users into flying.
- **Robust Build:** The SkyHawk Zephyr is constructed to withstand typical beginner mistakes, making it durable and reliable for novice pilots.
- **Photography and Videography:*

---

## Reset

After completing the query, it is recommended to clean up the environment by removing the graph you created. You can drop the graph by running the following single line of code.

In [9]:
>>> G.drop_graph()

2025-03-28 15:20:17,365 - tigergraphx.core.managers.schema_manager - INFO - Dropping graph: RetailGraph...
2025-03-28 15:20:22,112 - tigergraphx.core.managers.schema_manager - INFO - Graph dropped successfully.


---

## What’s Next?

- [API Reference](../reference/introduction.md): Dive deeper into TigerGraphX APIs.

---

Start transforming your GraphRAG workflows with the power of **TigerGraphX** today!